## The Battle of Neighborhoods (Part 2)
## Introduction: Business Problem 
In this project we will try to find an optimal location for a new services. Specifically, this report will be targeted to stakeholders interested in opening a point of service (shop, restaurant, bar) in Toronto, Canada.

Here we will try finding if someone wants to open a new shop in the city which location is best suited for it keeping in mind the competitors and which income group of people will be attracted most to it based on the population of the neighbourhood.

Since there are lots of shops in Toronto we will try to detect locations that are not already crowded with restaurants. We would also prefer locations as close to city center as possible, assuming that first two conditions are met.


## Step 1. Data Collection

Based on definition of our problem, factors that will influence our decission are:

All existing shops in the neighborhood (any type of shop)
Distance of neighborhood from city center
Age group of people with their income
Decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:

centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
number of shops and their type and location in every neighborhood will be obtained using Foursquare API

Link: https://ru.foursquare.com/explore?mode=url&ne=43.836013%2C-79.120274&q=%D0%9F%D0%BE%D0%BA%D1%83%D0%BF%D0%BA%D0%B8&sw=43.557506%2C-79.757824


In [1]:
# install packages
!conda install -c anaconda beautifulsoup4 --yes
!conda install -c anaconda lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         156 KB  anaconda

The following packages will be UPDATED:

    certifi: 2019.11.28-py36_0 conda-forge --> 2019.11.28-py36_0 anaconda
    openssl: 1.1.1d-h516909a_0 conda-forge --> 1.1.1-h7b6447c_0  anaconda


certifi-2019.11.28   | 156 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  curren

In [5]:
# import libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from urllib.request import urlopen #import the library used to query a website
from bs4 import BeautifulSoup #import the Beautiful soup functions to parse the data returned from the website

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)

# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 

# instantiate the dataframe
Nebr = pd.DataFrame(columns=column_names)

wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urlopen(wiki)

#Parse the html in the 'page' variable, and store it in Beautiful Soup format
soup = BeautifulSoup(page, "lxml")
Toronto=soup.find('table', class_='wikitable sortable')

In [7]:
# Make more comfortable lists
P=[]
B=[]
N=[]

for row in Toronto.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        P.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        N.append(cells[2].find(text=True))

        
#Adding Data to our DataFrame
Nebr['Postal_Code']=P
Nebr['Borough']=B
Nebr['Neighborhood']=N

Nebr

,Postal_Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned\n
9,M9A,Queen's Park,Not assigned\n


In [8]:
# Data Cleaning
# If Borough is Not Assigned drop row.
# Reset Index
Nebr = Nebr.drop(Nebr[Nebr['Borough'].str.contains("Not assigned")==True].index, axis=0, inplace=False)
Nebr.index = pd.RangeIndex(len(Nebr.index))
Nebr

,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned\n
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North\n


In [9]:
Nebr.shape

(210, 3)

In [10]:
column_n = ['NEBRVAL'] 
NEBR_NA = pd.DataFrame(columns=column_n)
NA=['Not assigned'] 
NEBR_NA['NEBRVAL'] = NA

Nebr1=Nebr

for row_index,row in Nebr.iterrows():
    if((Nebr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned') or (Nebr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned\n')):
       
        Nebr1.loc[row_index,['Neighborhood']] = Nebr1.loc[row_index,['Borough']].values.astype('str') 
        
#Reset Index
#nbr.index = pd.RangeIndex(len(nbr.index))

Nebr1

,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North\n


In [11]:
# 
Nebr2=Nebr1.groupby('Postal_Code').agg({'Borough':'first',
                               'Neighborhood': ', '.join}).reset_index()

column_names = ['Postal_Code','Borough', 'Neighborhood'] 
Nebr3 = pd.DataFrame(columns=column_names)

Nebr3 = Nebr2.drop(Nebr2[Nebr2['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
Nebr3.index = pd.RangeIndex(len(Nebr3.index))
Nebr3

,Postal_Code,Borough,Neighborhood
0,M4E,East Toronto,The Beaches
1,M4K,East Toronto,"The Danforth West\n, Riverdale"
2,M4L,East Toronto,"The Beaches West\n, India Bazaar"
3,M4M,East Toronto,Studio District\n
4,M4N,Central Toronto,Lawrence Park
5,M4P,Central Toronto,Davisville North\n
6,M4R,Central Toronto,North Toronto West\n
7,M4S,Central Toronto,Davisville\n
8,M4T,Central Toronto,"Moore Park, Summerhill East\n"
9,M4V,Central Toronto,"Deer Park, Forest Hill SE\n, Rathnelly, South ..."


In [12]:
# Final actions
column = ['Postal_Code','Borough', 'Neighborhood'] 
Nebr_ungrp = pd.DataFrame(columns=column_names)
Nebr_ungrp = Nebr1.drop(Nebr1[Nebr1['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)
Nebr_ungrp.index = pd.RangeIndex(len(Nebr_ungrp.index))
Nebr_ungrp

,Postal_Code,Borough,Neighborhood
0,M5A,Downtown Toronto,Harbourfront
1,M7A,Downtown Toronto,Queen's Park
2,M5B,Downtown Toronto,Ryerson\n
3,M5B,Downtown Toronto,Garden District\n
4,M5C,Downtown Toronto,St. James Town
5,M4E,East Toronto,The Beaches
6,M5E,Downtown Toronto,Berczy Park
7,M5G,Downtown Toronto,Central Bay Street\n
8,M6G,Downtown Toronto,Christie\n
9,M5H,Downtown Toronto,Adelaide\n


In [13]:
# Instal Geolocations utilities
!conda install -c conda-forge geopy --yes 
import time
from geopy.geocoders import Nominatim
from geopy.util import get_version
get_version()

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge

The following packages will be UPDATED:

    certifi: 2019.11.28-py36_0 anaconda --> 2019.11.28-py36_0 conda-forge

The following packages will be DOWNGRADED:

    openssl: 1.1.1-h7b6447c_0  anaconda --> 1.1.1d-h516909a_0 conda-forge


certifi-2019.11.28   | 149 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


'1.21.0'

In [14]:
geolocator = Nominatim(scheme='http', user_agent="ES1234")

for row_index, item in Nebr_ungrp.iterrows():
    
    list1 = Nebr_ungrp.loc[[row_index],['Neighborhood']].values.astype('str')
    loc = ' , Toronto, Ontario, Canada'
    list1.astype('str')
    list1 = np.append(list1, loc)
    latitude = None
    longitude = None
    location = None
    
    location = geolocator.geocode(list1 , limit = 15)
    time.sleep(5)
    if(location is not None):
        Nebr_ungrp.loc[Nebr_ungrp.index[row_index], 'Latitude'] = location.latitude
        Nebr_ungrp.loc[Nebr_ungrp.index[row_index], 'Longitude'] = location.longitude

In [15]:
import json 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Importing to use the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [16]:
print('We have {} boroughs and {} neighborhoods.'.format(
        len(Nebr_ungrp['Borough'].unique()),
        Nebr_ungrp.shape[0]
    )
)

Nebr_ungrp.dropna(inplace =True)
Nebr_ungrp.index = pd.RangeIndex(len(Nebr_ungrp.index))

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ES1234")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

We have 4 boroughs and 74 neighborhoods.
The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [17]:
Nebr_ungrp

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
1,M7A,Downtown Toronto,Queen's Park,43.659659,-79.390340
2,M5B,Downtown Toronto,Ryerson\n,43.658469,-79.378993
3,M5B,Downtown Toronto,Garden District\n,43.656500,-79.377114
4,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704
5,M4E,East Toronto,The Beaches,43.671024,-79.296712
6,M5E,Downtown Toronto,Berczy Park,43.647984,-79.375396
7,M6G,Downtown Toronto,Christie\n,43.664111,-79.418405
8,M5H,Downtown Toronto,Adelaide\n,43.650486,-79.379498
9,M5H,Downtown Toronto,King\n,43.648949,-79.377754


In [18]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Nebr_ungrp['Latitude'], Nebr_ungrp['Longitude'], Nebr_ungrp['Borough'], Nebr_ungrp['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Use FOURSQUARE API to explore the neighbourhood

In [19]:
CLIENT_ID = 'ITASAC2XZTAR10QW4DD1BUBEZQGDO0AUENLWS502GZJIKNMQ'
CLIENT_SECRET = 'Z40J2TL4Z0XORSDAPFMWY5ITIKMBSFCYCPIV01FLLN3L5WBW'
VERSION = '20200203' # Foursquare API version

print('Successfully Logged-In')

Successfully Logged-In


In [20]:
Nebr_ungrp.loc[0]
neighborhood_latitude = np.float(Nebr_ungrp.loc[0,['Latitude']].values)
neighborhood_longitude =  np.float(Nebr_ungrp.loc[0,['Longitude']].values)

## Now, let's get the top 100 venues within a radius of 500 meters.

<b>First, create the GET request URL. </b>

In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [22]:
results = requests.get(url).json()

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Harbour Square Park,Park,43.639253,-79.378395
1,Lake Ontario,Lake,43.638945,-79.379665
2,Harbourfront,Neighborhood,43.639526,-79.380688
3,Miku,Japanese Restaurant,43.641374,-79.377531
4,Natrel Pond/Rink,Skating Rink,43.638431,-79.382528


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## Step 2. Gathering information about Neighborhoods

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
toronto_venues = getNearbyVenues(names=Nebr_ungrp['Neighborhood'],
                                   latitudes=Nebr_ungrp['Latitude'],
                                   longitudes=Nebr_ungrp['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson

Garden District

St. James Town
The Beaches
Berczy Park
Christie

Adelaide

King

Richmond

Dovercourt Village
Dufferin

Harbourfront East

Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West

Riverdale
Design Exchange
Toronto Dominion Centre
Brockton

Exhibition Place
Parkdale Village
The Beaches West

India Bazaar
Commerce Court
Lawrence Park
Roselawn

Davisville North

Forest Hill North
High Park
The Junction South

The Annex
Yorkville
Parkdale
Roncesvalles
Davisville

Harbord

University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East

Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE

Rathnelly
South Hill
Summerhill West

CN Tower
Bathurst Quay

Harbourfront West

King and Spadina
South Niagara
Rosedale
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


In [28]:
print(toronto_venues.shape)
toronto_venues.head()

(3437, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.64008,-79.38015,Harbour Square Park,43.639253,-79.378395,Park
1,Harbourfront,43.64008,-79.38015,Lake Ontario,43.638945,-79.379665,Lake
2,Harbourfront,43.64008,-79.38015,Harbourfront,43.639526,-79.380688,Neighborhood
3,Harbourfront,43.64008,-79.38015,Miku,43.641374,-79.377531,Japanese Restaurant
4,Harbourfront,43.64008,-79.38015,Natrel Pond/Rink,43.638431,-79.382528,Skating Rink


In [29]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide\n,100,100,100,100,100,100
Bathurst Quay\n,24,24,24,24,24,24
Berczy Park,100,100,100,100,100,100
Brockton\n,18,18,18,18,18,18
CN Tower,85,85,85,85,85,85
Cabbagetown,52,52,52,52,52,52
Chinatown,90,90,90,90,90,90
Christie\n,56,56,56,56,56,56
Church and Wellesley,84,84,84,84,84,84


In [30]:
print('There are {} uniques categories in boroughs.'.format(len(toronto_venues['Venue Category'].unique())))

There are 292 uniques categories in boroughs.


## Step 3. Analyze.

In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport Service,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Storage Facility,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game

In [32]:
toronto_onehot.shape

(3437, 292)

In [33]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport Service,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Storage Facility,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game

In [34]:
toronto_grouped.shape

(63, 292)

In [37]:
# Check Top venues
Top_venues = 5

for hood in toronto_grouped['Neighborhood']:
#    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
 #   print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(Top_venues))
 #   print('\n')

In [38]:
def return_most_common_venues(row, Top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:Top_venues]

In [39]:
Top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(Top_venues):
    try:
        columns.append('{}{} Popular Venues'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Popular Venues'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], Top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
0,Adelaide\n,Coffee Shop,Café,Restaurant,Gastropub,Seafood Restaurant,Gym,American Restaurant,Hotel,Cosmetics Shop,Burger Joint
1,Bathurst Quay\n,Coffee Shop,Café,Park,Gym,Sushi Restaurant,Sculpture Garden,Caribbean Restaurant,Diner,Japanese Restaurant,Bank
2,Berczy Park,Coffee Shop,Café,Italian Restaurant,Restaurant,Hotel,Japanese Restaurant,Beer Bar,Seafood Restaurant,Bakery,Cocktail Bar
3,Brockton\n,Bar,Vietnamese Restaurant,Park,Pizza Place,French Restaurant,Dive Bar,Jazz Club,Bakery,Coffee Shop,Gastropub
4,CN Tower,Hotel,Coffee Shop,Pizza Place,Italian Restaurant,Restaurant,Aquarium,Baseball Stadium,Bar,Concert Hall,Gym
5,Cabbagetown,Coffee Shop,Café,Restaurant,Diner,Pizza Place,Indian Restaurant,Gastropub,Bakery,Beer Store,Pub
6,Chinatown,Vietnamese Restaurant,Café,Chinese Restaurant,Dessert Shop,Bar,Dumpling Restaurant,Coffee Shop,Mexican Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar
7,Christie\n,Korean Restaurant,Coffee Shop,Café,Mexican Restaurant,Dessert Shop,Indian Restaurant,Sandwich Place,Cocktail Bar,Grocery Store,Japanese Restaurant
8,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Café,Restaurant,Gym,Men's Store,Mediterranean Restaurant
9,Commerce Court,Coffee Shop,Café,Restaurant,Hotel,Gym,Japanese Restaurant,Italian Restaurant,American Restaurant,Steakhouse,Gastropub


## Step 4. K-Mean clustering

In [60]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
labels = kmeans.labels_[0:63] 
print(labels)

[0 0 0 6 9 9 6 7 9 0 9 9 9 0 0 6 9 0 3 3 9 9 7 0 0 0 9 6 6 0 9 9 6 1 9 9 0
 9 2 6 9 8 9 0 0 9 9 0 9 9 4 9 9 9 9 9 0 5 9 9 0 6 9]


In [61]:
toronto_merged = Nebr_ungrp
print(toronto_merged.shape)
labels = np.append(labels,labels[0])
print(labels.shape)
# add clustering labels
toronto_merged['Cluster Labels'] = labels.tolist()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

(64, 6)
(64,)


,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
0,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150,0,Coffee Shop,Café,Hotel,Pizza Place,Italian Restaurant,Restaurant,Gym,Sports Bar,Sporting Goods Shop,Steakhouse
1,M7A,Downtown Toronto,Queen's Park,43.659659,-79.390340,0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Ice Cream Shop,Chinese Restaurant,French Restaurant,Juice Bar,Thai Restaurant
2,M5B,Downtown Toronto,Ryerson\n,43.658469,-79.378993,0,Coffee Shop,Ramen Restaurant,Café,Burger Joint,Clothing Store,Japanese Restaurant,Tea Room,Restaurant,Pizza Place,Bakery
3,M5B,Downtown Toronto,Garden District\n,43.656500,-79.377114,6,Coffee Shop,Clothing Store,Restaurant,Hotel,Cosmetics Shop,Fast Food Restaurant,Japanese Restaurant,Bookstore,Bakery,Tea Room
4,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704,9,Coffee Shop,Pizza Place,Grocery Store,Café,Hostel,Park,Bistro,Breakfast Spot,Bus Stop,Caribbean Restaurant


In [62]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Step 5 Finalization

In [63]:
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
0,Downtown Toronto,0,Coffee Shop,Café,Hotel,Pizza Place,Italian Restaurant,Restaurant,Gym,Sports Bar,Sporting Goods Shop,Steakhouse
1,Downtown Toronto,0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Ice Cream Shop,Chinese Restaurant,French Restaurant,Juice Bar,Thai Restaurant
2,Downtown Toronto,0,Coffee Shop,Ramen Restaurant,Café,Burger Joint,Clothing Store,Japanese Restaurant,Tea Room,Restaurant,Pizza Place,Bakery
9,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,Seafood Restaurant,Gastropub,Japanese Restaurant,Gym,American Restaurant,Italian Restaurant
13,Downtown Toronto,0,Coffee Shop,Café,Hotel,Pizza Place,Italian Restaurant,Restaurant,Gym,Sports Bar,Sporting Goods Shop,Steakhouse
14,Downtown Toronto,0,Playground,American Restaurant,Beach,Beer Garden,Scenic Lookout,Farm,Pier,Filipino Restaurant,Festival,Fast Food Restaurant
17,West Toronto,0,Coffee Shop,Café,Museum,Restaurant,Park,Boutique,Clothing Store,College Auditorium,Jewelry Store,College Theater
23,West Toronto,0,Theme Park,Soccer Stadium,Arts & Crafts Store,Athletics & Sports,Music Venue,Plaza,Coffee Shop,Lounge,Theater,Racetrack
24,West Toronto,0,Tibetan Restaurant,Diner,Pharmacy,Bar,Café,Bakery,Restaurant,Indian Restaurant,Gift Shop,Donut Shop
25,East Toronto,0,Beach,Bar,Park,Japanese Restaurant,Coffee Shop,Tea Room,Thai Restaurant,Bakery,Breakfast Spot,Restaurant


In [64]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
33,West Toronto,1,Bar,Coffee Shop,Italian Restaurant,Café,Thai Restaurant,Mexican Restaurant,Grocery Store,Nail Salon,Antique Shop,Bakery


In [65]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
38,Central Toronto,2,Italian Restaurant,Sushi Restaurant,Convenience Store,Pub,Coffee Shop,Furniture / Home Store,Trail,Flower Shop,Mexican Restaurant,Japanese Restaurant


In [66]:
# Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
18,East Toronto,3,Coffee Shop,Pharmacy,Bus Line,Ice Cream Shop,Caribbean Restaurant,Café,Sushi Restaurant,Fried Chicken Joint,French Restaurant,Sandwich Place
19,East Toronto,3,Vietnamese Restaurant,Chinese Restaurant,Bakery,Bar,Light Rail Station,Grocery Store,Fast Food Restaurant,Baseball Field,Trail,Coffee Shop


In [67]:
# Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
50,Central Toronto,4,Mexican Restaurant,Park,French Restaurant,Italian Restaurant,Pet Store,Pub,Café,Flower Shop,Shoe Repair,Liquor Store


In [69]:
# Cluster 6
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
57,Downtown Toronto,5,Bakery,Dessert Shop,Pizza Place,Yoga Studio,Coffee Shop,Café,Furniture / Home Store,American Restaurant,Asian Restaurant,Middle Eastern Restaurant


In [70]:
# Cluster 7
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
3,Downtown Toronto,6,Coffee Shop,Clothing Store,Restaurant,Hotel,Cosmetics Shop,Fast Food Restaurant,Japanese Restaurant,Bookstore,Bakery,Tea Room
6,Downtown Toronto,6,Coffee Shop,Café,Italian Restaurant,Restaurant,Hotel,Japanese Restaurant,Beer Bar,Seafood Restaurant,Bakery,Cocktail Bar
15,Downtown Toronto,6,Coffee Shop,Café,Restaurant,Deli / Bodega,Hotel,Italian Restaurant,Japanese Restaurant,Salad Place,Sporting Goods Shop,Steakhouse
27,Downtown Toronto,6,Coffee Shop,Café,Restaurant,Hotel,Gym,Japanese Restaurant,Italian Restaurant,American Restaurant,Steakhouse,Gastropub
28,Central Toronto,6,Sushi Restaurant,Coffee Shop,Italian Restaurant,Bakery,Salon / Barbershop,Pub,Asian Restaurant,Burger Joint,Hobby Shop,Sandwich Place
32,West Toronto,6,Convenience Store,Pet Store,Pub,Metro Station,Pharmacy,Sporting Goods Shop,Food & Drink Shop,Sandwich Place,Tennis Court,Theme Park Ride / Attraction
39,Downtown Toronto,6,Korean Restaurant,Coffee Shop,Grocery Store,Café,Park,Sandwich Place,Mexican Restaurant,Pizza Place,Ice Cream Shop,Dessert Shop
61,Downtown Toronto,6,Coffee Shop,Café,Hotel,Gym,Restaurant,Gastropub,Steakhouse,Deli / Bodega,American Restaurant,Seafood Restaurant


In [71]:
# Cluster 8
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
7,Downtown Toronto,7,Korean Restaurant,Coffee Shop,Café,Mexican Restaurant,Dessert Shop,Indian Restaurant,Sandwich Place,Cocktail Bar,Grocery Store,Japanese Restaurant
22,West Toronto,7,Bar,Vietnamese Restaurant,Park,Pizza Place,French Restaurant,Dive Bar,Jazz Club,Bakery,Coffee Shop,Gastropub


In [72]:
# Cluster 9
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
41,West Toronto,8,Café,Coffee Shop,Bakery,Pizza Place,Pet Store,Restaurant,Dessert Shop,Tea Room,Shoe Store,Pharmacy


In [73]:
# Cluster 10
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
4,Downtown Toronto,9,Coffee Shop,Pizza Place,Grocery Store,Café,Hostel,Park,Bistro,Breakfast Spot,Bus Stop,Caribbean Restaurant
5,East Toronto,9,Beach,Bar,Park,Japanese Restaurant,Coffee Shop,Tea Room,Thai Restaurant,Bakery,Breakfast Spot,Restaurant
8,Downtown Toronto,9,Coffee Shop,Café,Restaurant,Gastropub,Seafood Restaurant,Gym,American Restaurant,Hotel,Cosmetics Shop,Burger Joint
10,Downtown Toronto,9,Beer Store,Ice Cream Shop,Women's Store,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
11,West Toronto,9,Café,Pizza Place,Park,Bus Line,Brazilian Restaurant,Coffee Shop,Restaurant,Bar,Deli / Bodega,Egyptian Restaurant
12,West Toronto,9,Bar,Café,Bakery,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Sandwich Place,Beer Bar,Cocktail Bar,Beer Store
16,West Toronto,9,Bar,Café,Restaurant,Coffee Shop,Korean Restaurant,Cocktail Bar,Bakery,Boutique,French Restaurant,Portuguese Restaurant
20,Downtown Toronto,9,Coffee Shop,Café,Restaurant,Hotel,Bakery,Deli / Bodega,Gym,American Restaurant,Seafood Restaurant,Steakhouse
21,Downtown Toronto,9,Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Steakhouse,Seafood Restaurant,Gastropub,Bar,Italian Restaurant
26,East Toronto,9,Indian Restaurant,Grocery Store,Café,Women's Store,Diner,Brewery,Snack Place,Bus Line,Shopping Plaza,Sandwich Place
